In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/kaggle/isl_data.csv')

# Split features and labels
features = data.iloc[:, :-1].values
labels = data.iloc[:, -1].values

labels = list(map(str, labels))
print(labels)

['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',

<ipython-input-3-1cbe2791e4cb>:1: DtypeWarning: Columns (126) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/kaggle/isl_data.csv')


In [4]:
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)


In [5]:
onehot_encoder = OneHotEncoder(sparse=False)
labels_onehot = onehot_encoder.fit_transform(labels_encoded.reshape(-1, 1))


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [6]:
X_train, X_test, y_train, y_test = train_test_split(features, labels_onehot, test_size=0.2, random_state=42)


In [7]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(126, 1)))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(256, activation='relu'))
model.add(Dense(35, activation='softmax'))

In [8]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [9]:
# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)


In [10]:
X_train = X_train.reshape((-1, 126, 1))
X_test = X_test.reshape((-1, 126, 1))

In [11]:
model.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1032/1032 [==============================] - 31s 19ms/step - loss: 2.1459 - accuracy: 0.3177 - val_loss: 1.0880 - val_accuracy: 0.6014
Epoch 2/100
1032/1032 [==============================] - 19s 18ms/step - loss: 0.7943 - accuracy: 0.7071 - val_loss: 0.4961 - val_accuracy: 0.8095
Epoch 3/100
1032/1032 [==============================] - 18s 17ms/step - loss: 0.4303 - accuracy: 0.8510 - val_loss: 0.2890 - val_accuracy: 0.9059
Epoch 4/100
1032/1032 [==============================] - 18s 17ms/step - loss: 0.3600 - accuracy: 0.8844 - val_loss: 0.1631 - val_accuracy: 0.9529
Epoch 5/100
1032/1032 [==============================] - 23s 22ms/step - loss: 0.1766 - accuracy: 0.9463 - val_loss: 0.1917 - val_accuracy: 0.9431
Epoch 6/100
1032/1032 [==============================] - 18s 17ms/step - loss: 0.1617 - accuracy: 0.9486 - val_loss: 0.1041 - val_accuracy: 0.9704
Epoch 7/100
1032/1032 [==============================] - 19s 19ms/step - loss: 0.1452 - accuracy: 0.9563 - val_loss: 0

In [12]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

258/258 [==============================] - 3s 13ms/step - loss: 0.0182 - accuracy: 0.9962
Test loss: 0.01818172074854374
Test accuracy: 0.9962451457977295


In [13]:
model.save("isl_neural_network.h5")